In [1]:
import os, platform, sys, time
from datetime import date
import cv2
import numpy as np
from collections import defaultdict
import face_recognition
import imutils
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Activation, AveragePooling2D
from PIL import Image
import matplotlib.pyplot as plt
#import screen_brightness_control as sbc

In [2]:
IMG_SIZE = 24

In [3]:
def predict(img, model):
    img = np.array(Image.fromarray(img, 'RGB').convert('L'))
    cv2.imshow("prediction" , img)
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE)).astype('float32')
    img /= 255
    img = img.reshape(1,IMG_SIZE,IMG_SIZE,1)
    prediction = model.predict(img)
    ##print(prediction)
    if prediction < 0.1:
        prediction = 'closed'
    elif prediction > 0.9:
        prediction = 'open'
    else:
        prediction = 'idk'
    return prediction

In [4]:
def init():
    face_cascPath = 'haarcascade_frontalface_alt.xml'
    open_eye_cascPath = "haarcascade_eye.xml"
    open_eyeglasses_cascPath = 'haarcascade_eye_tree_eyeglasses.xml'
    left_eye_cascPath = 'haarcascade_lefteye_2splits.xml'
    right_eye_cascPath ='haarcascade_righteye_2splits.xml'

    face_detector = cv2.CascadeClassifier(cv2.data.haarcascades+face_cascPath)
    open_eyes_detector = cv2.CascadeClassifier(cv2.data.haarcascades+open_eye_cascPath)
    open_eyesglasses_detector = cv2.CascadeClassifier(cv2.data.haarcascades+open_eyeglasses_cascPath)
    left_eye_detector = cv2.CascadeClassifier(cv2.data.haarcascades+left_eye_cascPath)
    right_eye_detector = cv2.CascadeClassifier(cv2.data.haarcascades+right_eye_cascPath)

    model = Sequential([Conv2D(6, (3,3), input_shape = (24,24,1)),
                       Activation("relu"),
                       AveragePooling2D(),
                       Conv2D(16, (3,3), input_shape = (24,24,1)),
                       Activation("relu"),
                       AveragePooling2D(),
                       Flatten(),
                       Dense(120),
                       Activation('relu'),
                       Dense(84),
                       Activation('relu'),
                       Dense(1),
                       Activation('sigmoid')
                      ])
    model.load_weights("eye_status.h5")


    print("[LOG] Collecting images ...")
    images = []
    for direc, _, files in tqdm(os.walk("ImagesTrain")):
        for file in files:
            if file.endswith("jpeg") or file.endswith("jpg"):
                images.append(os.path.join(direc,file))
    return (model,face_detector, open_eyes_detector, open_eyesglasses_detector, left_eye_detector, right_eye_detector, images) 

def process_and_encode(images):
    # initialize the list of known encodings and known names
    known_encodings = []
    known_names = []
    print("[LOG] Encoding faces ...")

    for image_path in tqdm(images):
        # Load image
        image = cv2.imread(image_path)
        # Convert it from BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
     
        # detect face in the image and get its location (square boxes coordinates)
        boxes = face_recognition.face_locations(image, model='hog')

        # Encode the face into a 128-d embeddings vector
        encoding = face_recognition.face_encodings(image, boxes)

        # the person's name is the name of the folder where the image comes from
        name = image_path.split(os.path.sep)[-1]

        if len(encoding) > 0 : 
            known_encodings.append(encoding[0])
            known_names.append(name)

    return {"encodings": known_encodings, "names": known_names}

In [5]:
def isBlinking(history, maxFrames):
    """ @history: A string containing the history of eyes status 
         where a '1' means that the eyes were closed and '0' open.
        @maxFrames: The maximal number of successive frames where an eye is closed """
    print(history)
    for i in range(maxFrames):
        pattern = '1' + '0'*(i+1) + '1'
        if pattern in history:
            return True
    return False

def detect_and_display(model, frame, face_detector, open_eyes_detector, open_eyesglasses_detector, left_eye_detector, right_eye_detector, data, eyes_detected):
        # resize the frame
        frame = cv2.resize(frame, (0, 0), fx=0.6, fy=0.6)
            
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Detect faces
        faces = face_detector.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(50, 50),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        # for each detected face
        for (x,y,w,h) in faces:
            # Encode the face into a 128-d embeddings vector
            encoding = face_recognition.face_encodings(rgb, [(y, x+w, y+h, x)])[0]

            # Compare the vector with all known faces encodings
            matches = face_recognition.compare_faces(data["encodings"], encoding)

            # For now we don't know the person name
            name = "Unknown"

            # If there is at least one match:
            if True in matches:
                matchedIdxs = [i for (i, b) in enumerate(matches) if b]
                counts = {}
                for i in matchedIdxs:
                    name = data["names"][i]
                    counts[name] = counts.get(name, 0) + 1

                # determine the recognized face with the largest number of votes
                name = max(counts, key=counts.get)

            face = frame[y:y+h,x:x+w]
            gray_face = gray[y:y+h,x:x+w]

            eyes = []
            
            # Eyes detection
            # check first if eyes are open (with glasses taking into account)
            '''open_eyes_glasses = open_eyesglasses_detector.detectMultiScale(
                gray_face,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(30, 30),
                flags = cv2.CASCADE_SCALE_IMAGE
            )
            
            open_eyes = open_eyes_detector.detectMultiScale(
                gray_face,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(30, 30),
                flags = cv2.CASCADE_SCALE_IMAGE
            )
            '''
            # if open_eyes_glasses detect eyes then they are open 
            open_eyes_glasses = []
            open_eyes = []
            if len(open_eyes_glasses) == 2 or len(open_eyes)==2:
                eyes_detected[name]+='1'
                for (ex,ey,ew,eh) in open_eyes_glasses:
                    cv2.rectangle(face,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            
            # otherwise try detecting eyes using left and right_eye_detector
            # which can detect open and closed eyes                
            else:
                # separate the face into left and right sides
                left_face = frame[y:y+h, x+int(w/2):x+w]
                left_face_gray = gray[y:y+h, x+int(w/2):x+w]

                right_face = frame[y:y+h, x:x+int(w/2)]
                right_face_gray = gray[y:y+h, x:x+int(w/2)]

                # Detect the left eye
                left_eye = left_eye_detector.detectMultiScale(
                    left_face_gray,
                    scaleFactor=1.1,
                    minNeighbors=5,
                    minSize=(30, 30),
                    flags = cv2.CASCADE_SCALE_IMAGE
                )

                # Detect the right eye
                right_eye = right_eye_detector.detectMultiScale(
                    right_face_gray,
                    scaleFactor=1.1,
                    minNeighbors=5,
                    minSize=(30, 30),
                    flags = cv2.CASCADE_SCALE_IMAGE
                )
                eye_status = '1'
                closed = True # we suppose the eyes are open

                # For each eye check wether the eye is closed.
                # If one is closed we conclude the eyes are closed
                for (ex,ey,ew,eh) in right_eye:
                    color = (0,255,0)
                    cv2.imshow("eye_right", right_face[ey:ey+eh,ex:ex+ew] )
                    pred = predict(right_face[ey:ey+eh,ex:ex+ew],model)
                    if pred == 'closed': 
                        print('right closed')
                        color = (0,0,255)
                    else:
                        closed = False
                    cv2.rectangle(right_face,(ex,ey),(ex+ew,ey+eh),color,2)
                for (ex,ey,ew,eh) in left_eye:
                    color = (0,255,0)
                    cv2.imshow("eye_left", left_face[ey:ey+eh,ex:ex+ew] )
                    pred = predict(left_face[ey:ey+eh,ex:ex+ew],model)
                    if pred == 'closed':
                        print('left closed')
                        color = (0,0,255)
                    else:
                        closed = False
                    cv2.rectangle(left_face,(ex,ey),(ex+ew,ey+eh),color,2)
                if closed:
                    eye_status = '0'
                eyes_detected[name] += eye_status
            # Each time, we check if the person has blinked
            # If yes, we display its name
            print(name ,eyes)
            if isBlinking(eyes_detected[name],3):
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                # Display name
                y = y - 15 if y - 15 > 15 else y + 15
                cv2.putText(frame, name+"is blinking", (x, y), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)
                #plt.imshow(frame)
                #plt.show()
        return frame

In [6]:
(model, face_detector, open_eyes_detector,open_eyesglasses_detector, left_eye_detector, right_eye_detector, images) = init()
data = process_and_encode(images)

[LOG] Collecting images ...


1it [00:00, 686.92it/s]


[LOG] Encoding faces ...


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:10<00:00,  2.04s/it]


In [7]:
eyes_detected = defaultdict(str)
cap = cv2.VideoCapture(0)
#initial_brightness = sbc.get_brightness()
#sbc.set_brightness(75)
while True:
    ret , frame = cap.read()
    #print("frame")
    frame = detect_and_display(model, frame, face_detector, open_eyes_detector,open_eyesglasses_detector, left_eye_detector, right_eye_detector, data, eyes_detected)
    #plt.imshow(frame)
    cv2.imshow("Live face detector" , frame)
    #plt.show()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
cap.release()
#sbc.set_brightness(initial_brightness)

frame
frame
[[0.9303294]]
[[0.21622992]]
Kopal.jpg []
1
frame
[[0.01604348]]
right closed
[[0.02952766]]
left closed
Kopal.jpg []
10
frame
[[0.4455178]]
[[0.15302226]]
Kopal.jpg []
101
frame
[[0.5711021]]
[[0.14863414]]
Kopal.jpg []
1011
frame
[[0.0073486]]
right closed
[[0.00803125]]
left closed
Kopal.jpg []
10110
frame
[[0.5685756]]
[[0.09137109]]
left closed
Kopal.jpg []
101101
frame
[[0.6409192]]
[[0.4221521]]
Kopal.jpg []
1011011
frame
[[0.87869763]]
[[0.37168053]]
Kopal.jpg []
10110111
frame
[[0.14915228]]
[[0.06827685]]
left closed
Kopal.jpg []
101101111
frame
[[0.5095161]]
[[0.23230046]]
Kopal.jpg []
1011011111
frame
[[0.74268717]]
[[0.21533686]]
Kopal.jpg []
10110111111
frame
[[0.00650111]]
right closed
[[0.00488302]]
left closed
Kopal.jpg []
101101111110
frame
[[0.4950057]]
[[0.51001304]]
Kopal.jpg []
1011011111101
frame
[[0.49951065]]
[[0.20999613]]
Kopal.jpg []
10110111111011
frame
[[0.826674]]
[[0.6120563]]
Kopal.jpg []
101101111110111
frame
[[0.78941315]]
[[0.2866348]]
Ko